In [1]:
pip list

Package                            Version
---------------------------------- -----------------
absl-py                            0.15.0
alabaster                          0.7.12
alembic                            1.7.6
anaconda-client                    1.7.2
anaconda-navigator                 2.0.3
anaconda-project                   0.9.1
ansicolors                         1.1.8
anyio                              2.2.0
appdirs                            1.4.4
applaunchservices                  0.2.1
appnope                            0.1.2
appscript                          1.1.2
argh                               0.26.2
argon2-cffi                        20.1.0
arrow                              1.2.2
arviz                              0.12.1
asn1crypto                         1.4.0
asteval                            0.9.25
astroid                            2.5
astropy                            4.2.1
astunparse                         1.6.3
async-generator                    1.10

In [21]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import plotly.graph_objects as go
import plotly.io as pio
import requests
from lmfit import minimize, Parameters, Parameter, report_fit
pio.renderers.default = "notebook"
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
#plt.style.use('ggplot')


In [22]:
from sklearn import metrics
from sklearn.metrics import r2_score#R square

In [23]:
import seaborn as sns
sns.set_context('notebook')
sns.set_style('white')

In [24]:
# Jupyter Specifics
from IPython.display import HTML
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout, ToggleButton, ToggleButtons

style = {'description_width': '100px'}
slider_layout = Layout(width='99%')

In [25]:
def ode_model(z, t,  beta, beta1, beta2,  k1,k2, deta1, deta2, gama1,gama2,alpha1,yita1,yita2):
    """
    Reference https://www.idmod.org/docs/hiv/model-seir.html
    """

    S,S1, E_0,E_V, I_0,I_V, R_0,R_V,R3,R4,D,I_C= z
    N = S+S1+ E_0+E_V+ I_0+I_V+D+R_0+R_V+R3+R4+D
    #IPV=beta*S*I/N+beta1*S*P/N+beta2*S*V/N
    dSdt = -beta*S*I_0/N-beta1*S*I_V/N-alpha1*S
    dE_0dt = beta*S*I_0/N+beta*R4*I_0/N - k1*E_0
    dI_Cdt=k1*E_0+k2*E_V
    dI_0dt = k1*E_0 - (gama1+deta1)*I_0
    dR_0dt = gama1*I_0-yita2*R_0
    dR3dt = yita2*R_0-beta1*R3*I_V/N-alpha1*R3
    dS1dt = alpha1*(S+R3+R4)-beta2*(S1-alpha1*R4)*I_V/N
    dE_Vdt = beta1*S*I_V/N+beta2*(S1-alpha1*R4)*I_V/N+beta1*R3*I_V/N - k2*E_V
    dI_Vdt=k2*E_V-(deta2+gama2)*I_V
    dR4dt =yita1*R_V-beta*R4*I_0/N-alpha1*R4
    dR_Vdt=gama2*I_V-yita1*R_V
    dDdt=deta1*I_0+deta2*I_V
    return [dSdt,dS1dt, dE_0dt,dE_Vdt, dI_0dt,dI_Vdt, dR_0dt,dR_Vdt,dR3dt,dR4dt,dDdt,dI_Cdt]

In [26]:
def ode_solver(t, initial_conditions, params):
    initS1,initE_0,initE_V, initI_0,initI_V,initR_0,initR_V,initR3,initR4,  initN,  initD,initI_C= initial_conditions
    beta, beta1, beta2, k1,k2, deta1, deta2,gama1,gama2,alpha1,yita1,yita2 = params['beta'].value,params['beta1'].value,params['beta2'].value,params['k1'].value,params['k2'].value,params['deta1'].value,params['deta2'].value,params['gama1'].value,params['gama2'].value,params['alpha1'].value,params['yita1'].value,params['yita2'].value
    initS = initN-initS1 - initE_0-initE_V-initI_0-initI_V-initR_0-initR_V-initR3-initR4-initD
    
    res = odeint(ode_model, [initS,initS1, initE_0,initE_V,initI_0,initI_V,initR_0,initR_V,initR3,initR4,initD,initI_C], t, args=(beta, beta1, beta2, k1, k2,deta1, deta2,gama1, gama2,alpha1,yita1,yita2))
    return res

In [27]:
data1 = pd.read_excel('/Volumes/TOSHIBA EXT/percountrydata/Malaysia daily cases.xlsx')
z=data1['cases']
x=data1['cumulative_cases']



In [28]:
# ref: https://www.medrxiv.org/content/10.1101/2020.04.01.20049825v1.full.pdf
initN = 5900000
initI_C = 0
# S0 = 966000000
initS1 = 1
initE_0 = 12892
initE_V = 0
initI_0 = 35063
initI_V = 0

initR_0 = 885492
initR_V = 0
initR3 = 1000
initR4 = 0
initD = 27670
#[150]
'''initI_C = 3406220.321270449
# S0 = 966000000
initS1 = 54050.0767721145
initE_0 = 1571.399983439234
initE_V = 1
initI_0 = 29173.521769807463
initI_V = 0

initR_0 = 3895131.125931305
initR_V = 0
initR3 = 1258988.3378110304
initR4 = 0
initD = 212517.14534600038'''
#[75]
'''initI_C = 1374597.423005003
# S0 = 966000000
initS1 = 26446.27304186785
initE_0 = 157385.71788471864
initE_V = 1
initI_0 = 641995.3224658839
initI_V = 0

initR_0 = 1527162.741785394
initR_V = 0
initR3 = 1158447.174365456
initR4 = 0
initD = 74083.85879002837'''

#[50]
'''initI_C = 552991.6215057792
# S0 = 966000000
initS1 = 18687.954562057315
initE_0 = 97408.22635905513
initE_V = 1
initI_0 = 312414.6620205938
initI_V = 0

initR_0 = 1097029.0191932812
initR_V = 0
initR3 = 1128560.4739228918
initR4 = 0
initD = 44335.48882768001'''
#[100]
'''initN = 5900000
initI_C = 2324853.8686410105
# S0 = 966000000
initS1 = 32385.17003757665
initE_0 = 131170.76574474003
initE_V = 1
initI_0 = 808791.1734213082
initI_V = 0

initR_0 = 1097029.0191932812
initR_V = 0
initR3 = 2216765.7296132245
initR4 = 0
initD = 121452.73016834013 '''
#[10]
'''initN = 5900000
initI_C = 40167.92818593033
# S0 = 966000000
initS1 = 3886.9321478198303
initE_0 = 19923.908289204428
initE_V = 1
initI_0 = 56516.4031626314
initI_V = 0

initR_0 = 894183.5170458808
initR_V = 0
initR3 = 1093522.9218565081 
initR4 = 0
initD = 28801.500339332655 '''

beta = 0.16222299
beta1 = 0.33
beta2 = 4.92085281
k1 = 0.25000000
k2 = 0.37000000
deta1 = 0.00251122
deta2 =  1.4733e-05

gama1 = 0.03902327
gama2 = 0.03043903
alpha1=7.8989e-05
yita1= 0.001
yita2= 0.001
days =300

params = Parameters()
params.add('beta', value=beta, min=0, max=5)
params.add('beta1', value=beta1, min=0, max=1)
params.add('beta2', value=beta2, min=0, max=5)

params.add('k1', value=k1, min=0.25, max=0.37)
params.add('k2', value=k2, min=0.25, max=0.37)
params.add('deta1', value=deta1, min=0, max=0.01)
params.add('deta2', value=deta2, min=0, max=0.01)

params.add('gama1', value=gama1, min=0.03, max=0.3)
params.add('gama2', value=gama2, min=0.03, max=0.3)
params.add('alpha1', value=alpha1, min=0, max=0.00008)
params.add('yita1', value=yita1, min=0, max=0.01)
params.add('yita2', value=yita2, min=0, max=0.001)


In [ ]:
# ref: https://www.medrxiv.org/content/10.1101/2020.04.01.20049825v1.full.pdf
initN = 5900000
initI_C = 0
# S0 = 966000000
initS1 = 1
initE_0 = 12892
initE_V = 1
initI_0 = 35063
initI_V = 0
initR_0 = 885492
initR_V = 0
#initR3 = 1085492
initR4 = 0
initD = 27670



for yita2 in np.arange(0,0.001,0.0001):
    for initR3 in np.arange(0,3000000,300000):
        beta = 0.16222299
        beta1 = 0.28
        beta2 = 4.92085281
        k1 = 0.25000000
        k2 = 0.37000000
        deta1 = 0.00251122
        deta2 =  1.4733e-05

        gama1 = 0.03902327
        gama2 = 0.03043903
        alpha1=7.8989e-05
        yita1=1.8067e-05
        #yita2= 5.1625e-07
        days =240
        initial_conditions = [initS1,initE_0,initE_V, initI_0,initI_V, initR_0,initR_V,initR3,initR4, initN ,initD,initI_C]
        params['beta'].value,params['beta1'].value,params['beta2'].value,params['k1'].value,params['k2'].value,params['deta1'].value,params['deta2'].value,params['gama1'].value,params['gama2'].value,params['alpha1'].value,params['yita1'].value,params['yita2'].value = [beta, beta1, beta2,  k1,k2, deta1, deta2, gama1,gama2,alpha1,yita1,yita2]

        tspan = np.arange(0, days, 1)

        sol = ode_solver(tspan, initial_conditions, params)
        S,S1,E_0,E_V, I_0,I_V, R_0 ,R_V ,R3 ,R4, D, I_C= sol[:, 0], sol[:, 1], sol[:, 2], sol[:, 3], sol[:, 4], sol[:, 5], sol[:, 6], sol[:, 7], sol[:, 8],sol[:, 9],sol[:, 10],sol[:, 11]
        print(sum(k1*E_0)/sum(k2*E_V))
        #print(sum(k2*E_V))
        '''for t in range (0,4200):
            if (np.max(k2*E_V[300:4200]))>500:
                print(np.sum(k2*E_V[300:4200]))
            else:
                print(0)
            break'''
        #print(np.sum(k1*E_0[230:3000]))
            #print(gama1)
            #print(sum(k2*E_V))
            #print(np.argmax(k2*E_V))
        
        #print(sum(k2*E_V[300:2500]))
        '''for t in range (300,200):
            if k2*E_V[t]/(k2*E_V[t]+k1*E_0[t])>0.95:
                print(np.array(t))
                break
        for t in range (0,len(I_C)):
            print(type(I_C))
        for t in range (250,2500):
            if k2*E_V[t]>1000:
                print(np.array(t))
                break
        Q=[]
        for t in range (250,3600):
            if k2*E_V[t]>max(k2*E_V[t-5:t]) and k2*E_V[t]>max(k2*E_V[t+1:t+5]) and k2*E_V[t]>2000:
                Q.append(t)
                print(Q)
                
                if len(Q)>1:
                    print(Q[1]-Q[0])
                    break'''

                
    
params = Parameters()
params.add('beta', value=beta, min=0, max=5)
params.add('beta1', value=beta1, min=0, max=1)
params.add('beta2', value=beta2, min=0, max=5)

params.add('k1', value=k1, min=0.25, max=0.37)
params.add('k2', value=k2, min=0.25, max=0.37)
params.add('deta1', value=deta1, min=0, max=0.01)
params.add('deta2', value=deta2, min=0, max=0.01)

params.add('gama1', value=gama1, min=0.03, max=0.3)
params.add('gama2', value=gama2, min=0.03, max=0.3)
params.add('alpha1', value=alpha1, min=0, max=0.00008)
params.add('yita1', value=yita1, min=0, max=0.01)
params.add('yita2', value=yita2, min=0, max=0.001)


In [29]:
# ref: https://www.medrxiv.org/content/10.1101/2020.04.01.20049825v1.full.pdf
initN = 32800000

initI_C = 2476268
# S0 = 966000000
initS1 = 1
initE_0 = 19916
initE_V = 0
initI_0 = 67969
initI_V = 1
initR_0 = 2374761
initR_V = 0
initR3 = 0
initR4 = 0
initD = 28912

beta = 0.32464179
beta1 = 0.94126115
beta2 =0.17775162
k1 = 0.25
k2 = 0.37
deta1 = 0.05140007
deta2 = 0.00999366

gama1 = 0.30796859
gama2 = 0.38985876
alpha1=0.00913774
yita1=0.04357119 
yita2= 0.55744079
days =178

params = Parameters()
params.add('beta', value=beta, min=0.1, max=5)
params.add('beta1', value=beta1, min=0.5, max=5)
params.add('beta2', value=beta2, min=0.1, max=5)

params.add('k1', value=k1, min=0.15, max=0.25)
params.add('k2', value=k2, min=0.3, max=0.37)
params.add('deta1', value=deta1, min=0, max=0.1)
params.add('deta2', value=deta2, min=0, max=0.01)

params.add('gama1', value=gama1, min=0.05, max=0.4)
params.add('gama2', value=gama2, min=0.05, max=0.4)
params.add('alpha1', value=alpha1, min=0, max=0.1)
params.add('yita1', value=yita1, min=0, max=0.2)
params.add('yita2', value=yita2, min=0.2, max=1)


In [30]:
def main(initS1,initE_0,initE_V, initI_0,initI_V, initR_0,initR_V, initR3,initR4, initN, initD  ,initI_C  ,beta, beta1,beta2, k1, k2, deta1, deta2, gama1,gama2,alpha1,yita1,yita2, days, param_fitting):
    initial_conditions = [initS1,initE_0,initE_V, initI_0,initI_V, initR_0,initR_V,initR3,initR4, initN ,initD,initI_C]
    params['beta'].value,params['beta1'].value,params['beta2'].value,params['k1'].value,params['k2'].value,params['deta1'].value,params['deta2'].value,params['gama1'].value,params['gama2'].value,params['alpha1'].value,params['yita1'].value,params['yita2'].value = [beta, beta1, beta2,  k1,k2, deta1, deta2, gama1,gama2,alpha1,yita1,yita2]
    
    tspan = np.arange(0, days, 1)
    
    sol = ode_solver(tspan, initial_conditions, params)

   
    '''print('MSE',np.square(np.subtract(x[120:127],(sol[:, 11][120:127]))).mean())
    print('RMSE: ', np.sqrt(np.mean((x[120:127]- (sol[:, 11][120:127]))**2)))
    print('RMAE',np.sqrt(metrics.mean_squared_error(x[120:127], (sol[:, 11][120:127]))))
    
    print( 'MAE',np.mean(np.abs(x[120:127]- (sol[:, 11][120:127]))))
    print('MAPE',mape(x[120:127], (sol[:, 11][120:127])))
    print('SMAPE',smape(x[120:127],(sol[:, 11][120:127])))
    print('R2',r2_score(x[120:127],(sol[:, 11][120:127])))''' 
    '''for i in range(0, len(tspan)):
        
        if tspan[i]<50:
            params['alpha1'].value=0
        else:
            params['alpha1'].value=0.1''' 


    #print(alpha1)
    
    S,S1,E_0,E_V, I_0,I_V, R_0 ,R_V ,R3 ,R4, D, I_C= sol[:, 0], sol[:, 1], sol[:, 2], sol[:, 3], sol[:, 4], sol[:, 5], sol[:, 6], sol[:, 7], sol[:, 8],sol[:, 9],sol[:, 10],sol[:, 11]
    
    #print(I_V/(I_0+I_V))
    
    #print(max(I_0))
    '''temp = []
    i = 0
    cout = 0
    for t in range(0, len(I_V)):
        cout += 1
        if cout < 14:
            i += (I_V[t]/(I_V[t]+I_0[t]))
        if cout == 14:
            i += (I_V[i]/(I_V[i]+I_0[i]))
            temp.append(i)
            cout = 0
            i = 0
    G = np.array(temp)
    print(G)''' 
    
    temp = []
    i = 0
    cout = 0
    for t in range(0, len(I_V)):
        cout += 1
        if cout < 14:
            i += (I_V[t]+I_0[t])
        if cout == 14:
            i += (I_V[t]+I_0[t])
            temp.append(i)
            cout = 0
            i = 0
    Qemp = []
    j = 0
    cout = 0
    for m in range(0, len(I_V)):
        cout += 1
        if cout < 14:
            j += (I_V[m])
        if cout == 14:
            j += (I_V[m])
            Qemp.append(j)
            cout = 0
            j = 0
    '''G = np.array(temp)
    M = np.array(Qemp)'''
    

    '''Qemp = []
    j = 0
    cout = 0
    for m in range(0, len(sol[:, 6])):
        cout += 1
        if cout < 14:
            j += (sol[:, 6][m])
        if cout == 14:
            j += (sol[:, 6][m])
            Qemp.append(j)
            cout = 0
            j = 0'''
    
    G = np.array(temp)
    M = np.array(Qemp)
    
    #print(M/G)
    
    '''qemp = []
    j = 0
    cout = 0
    for m in range(0, len(I_V)):
        cout += 1
        if cout < 14:
            j += (I_V[m])
        if cout == 14:
            j += (I_V[m])
            temp.append(j)
            cout = 0
            j = 0
    M = np.array(qemp)
    print(M)'''   
    
    
    #df = pd.DataFrame (I_C)
    #print(k1*E_0+k2*E_V)
    #filepath = '/Users/mac/Desktop/trans.xlsx'
    #df.to_excel(filepath, index=False)
  # Create traces
    fig = go.Figure()
    



    
    if not param_fitting:
        #fig.add_trace(go.Scatter(x=tspan, y=S, mode='lines+markers',name='Susceptible'))
        #fig.add_trace(go.Scatter(x=tspan, y=S1, mode='lines+markers',name='Susceptible1'))
        #fig.add_trace(go.Scatter(x=tspan, y=E_0, mode='lines+markers', name='Exposed_0'))
        #fig.add_trace(go.Scatter(x=tspan, y=E_V, mode='lines+markers', name='Exposed_V'))
        #fig.add_trace(go.Scatter(x=tspan, y=I_0, mode='lines+markers', name='Infected0'))
        fig.add_trace(go.Scatter(x=tspan, y=k1*E_0, mode='lines+markers', name='delta'))
        fig.add_trace(go.Scatter(x=tspan, y=k2*E_V, mode='lines+markers', name='Omicron'))
        #fig.add_trace(go.Scatter(x=tspan, y=k2*E_V, mode='lines+markers', name='Omicron'))
        #fig.add_trace(go.Scatter(x=tspan, y=R3, mode='lines+markers',name='Recovered0S'))
        #fig.add_trace(go.Scatter(x=tspan, y=R_0, mode='lines+markers',name='Recovered0'))
        #fig.add_trace(go.Scatter(x=tspan, y=R_V, mode='lines+markers',name='RecoveredV'))
        #fig.add_trace(go.Scatter(x=tspan, y=R4, mode='lines+markers',name='RecoveredVS'))
    #fig.add_trace(go.Scatter(x=tspan, y=D, mode='lines',name='mj'))
    fig.add_trace(go.Scatter(x=tspan, y=k1*E_0, mode='lines', name='Fitting', line = dict ( width = 5 , color = 'lightcoral' )))
    fig.add_trace(go.Scatter(x=tspan, y=I_C, mode='lines', name='Fitting',line = dict ( width = 5 , color = 'lightcoral' )))
        #
        #
        
        
    
        
        
    #fig.add_trace(go.Scatter(x=tspan, y=k1*E_0+k2*E_V, mode='lines+markers',line = dict(width=1,color='black'),name='Fitting curve',marker=dict(color='white',size = 8 , line = dict ( width = 1 , color = 'black'))))
        

    '''fig.add_trace(go.Scatter(x=tspan, y=D, mode='lines+markers',line = dict(width=1,color='black'),name='Fitting curve',marker=dict(color='white', size = 8 , line = dict ( width = 1 , color = 'black'))))'''
      
    if param_fitting:
        fig.add_trace(go.Scatter(x=tspan, y=x, mode='markers',\
                             name='Real data', line = dict(dash='dash')))
        fig.update_traces ( marker = dict ( size = 8 ,color='LightSkyBlue', line = dict ( width = 1 , color = 'black' )),selector=dict(mode='markers'))
        fig.add_trace(go.Scatter(x=tspan, y=z, mode='markers',\
                             name='Real data', line = dict(dash='dash'),marker=dict ( color='LightSkyBlue',
        size = 8 , line = dict ( width = 1 , color = 'black' ))))
        #fig.add_trace(go.Scatter(x=tspan, y=m, mode='lines', name='cumu',line = dict ( width = 5 , color = 'orange' )))
    
        
    if days <= 30:
        step = 1
    elif days <= 90:
        step = 7
    else:
        step = 31
    
    # Edit the layout
    fig.update_layout(font=dict(size=20),
                        
                       xaxis_title='Day',
                       #yaxis_title='Number of New Cases',
                       yaxis_title='Total Cases',
                       title_x=0.5,width=900, height=600, template = 'simple_white',legend=dict(font=dict(size=15),yanchor="top",y=0.99,xanchor="left",x=0.01,bordercolor="Black",
        borderwidth=2) #"plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none",y=0.99,x=0.01,y=0.65,x=0.83
)
    fig.update_xaxes(showgrid=True,showline=True,linewidth=2,mirror=True,tickangle=-45,tickfont={'size': 20},tickfont_family="sans-serif",ticktext=['<b>29 Nov 2021</b>', '<b>30 Dec 2021</b>','<b>30 Jan 2022</b> ', '<b>2 Mar 2022</b>','<b>2 Apr 2022</b>', '<b>3 May 2022</b>'],tickformat = None, tickmode='array', tickvals=np.arange(0, days + 1, step))
    fig.update_yaxes(showgrid=True, showline=True, mirror=True,tickfont={'size': 20},tickfont_family="sans-serif",linewidth=2)
    
    '''if not os.path.exists("images"):
        os.mkdir("images")'''

    #fig.write_image("images/S.2daily.JPG",scale=2)
    
   # ax.spines['bottom'].set_linewidth('2.0')#设置边框线宽为2.0




    #fig.write_image("images/m.png", dpi=900,format="png")
    #plt.savefig("S.K.1.jpg", dpi=1000, bbox_inches=None, pad_inches=0.1,)


    fig.show()




In [31]:
import numpy as np
from sklearn import metrics
from sklearn.metrics import r2_score#R square
def mape(z, upcoming_days_prediction):
    return np.mean(np.abs((z - upcoming_days_prediction) / z)) * 100
 
def smape(z, upcoming_days_prediction):
    return 2.0 * np.mean(np.abs(z - upcoming_days_prediction) / (np.abs(z) + np.abs(upcoming_days_prediction))) * 100

In [32]:
interact(main,
        initS1=IntSlider(min=0, max=100000, step=1, value=initS1, description='initS1', style=style, layout=slider_layout),
        initI_C=IntSlider(min=0, max=100000000, step=100, value=initI_C, description='initI_C', style=style, layout=slider_layout),
          
        initE_0=IntSlider(min=0, max=10000000, step=1, value=initE_0, description='initE_0', style=style, layout=slider_layout),
        initE_V=IntSlider(min=0, max=100000, step=1, value=initE_V, description='initE_V', style=style, layout=slider_layout),
        
        initI_0=IntSlider(min=0, max=100000000, step=10, value=initI_0, description='initI_0', style=style, layout=slider_layout),
        initI_V=IntSlider(min=0, max=100000, step=10, value=initI_V, description='initI_V', style=style, layout=slider_layout),
        
        initR_0=IntSlider(min=0, max=10000000, step=10, value=initR_0, description='initR_0', style=style, layout=slider_layout),
        initR_V=IntSlider(min=0, max=100000, step=10, value=initR_V, description='initR_V', style=style, layout=slider_layout),
        initR3=IntSlider(min=0, max=10000000, step=100, value=initR3, description='initR_0S', style=style, layout=slider_layout),
        initR4=IntSlider(min=0, max=10000000, step=100, value=initR4, description='initR_VS', style=style, layout=slider_layout),
        
        initD=IntSlider(min=0, max=100000, step=10, value=initD, description='initD', style=style, layout=slider_layout),
        initN=IntSlider(min=0, max=320000000, step=1000, value=initN, description='initN', style=style, layout=slider_layout),
        #initIPV=IntSlider(min=0, max=100000, step=10, value=initN, description='initIPV', style=style, layout=slider_layout),
        beta=FloatSlider(min=0, max=4, step=0.01, value=beta, description='Infection rate', style=style, layout=slider_layout),
        beta1=FloatSlider(min=0, max=14, step=0.01, value=beta1, description='beta1', style=style, layout=slider_layout),
        beta2=FloatSlider(min=0, max=4, step=0.01, value=beta2, description='beta2', style=style, layout=slider_layout),
        
        k1=FloatSlider(min=0, max=1, step=0.01, value=k1, description='k1', style=style, layout=slider_layout),
        k2=FloatSlider(min=0, max=1, step=0.01, value=k2, description='k2', style=style, layout=slider_layout),
        deta1=FloatSlider(min=0, max=1, step=0.01, value=deta1, description='deta1', style=style, layout=slider_layout),
        deta2=FloatSlider(min=0, max=1, step=0.01, value=deta2, description='deta2', style=style, layout=slider_layout),
        
        gama1=FloatSlider(min=0, max=1, step=0.01, value=gama1, description='gama1', style=style, layout=slider_layout),
        gama2=FloatSlider(min=0, max=1, step=0.01, value=gama2, description='gama2', style=style, layout=slider_layout),
        alpha1=FloatSlider(min=0, max=0.1, step=0.001, value=alpha1, description='alpha1', style=style, layout=slider_layout),
        yita1=FloatSlider(min=0, max=0.1, step=0.00001, value=yita1, description='yita1', style=style, layout=slider_layout),
        yita2=FloatSlider(min=0, max=0.001, step=0.0001, value=yita2, description='yita2', style=style, layout=slider_layout),
        
        days=IntSlider(min=1, max=5000, step=1, value=days, description='Days', style=style, layout=slider_layout),
        param_fitting=ToggleButton(value=False, description='Fitting Mode', disabled=False, button_style='', \
            tooltip='Click to show fewer plots', icon='check-circle')
              );
        
        

               

interactive(children=(IntSlider(value=1, description='initS1', layout=Layout(width='99%'), max=100000, style=S…

In [15]:
def error(params, initial_conditions, tspan, data):
    sol = ode_solver(tspan, initial_conditions, params)

    
    #return (sol_ - data).ravel()
    return (k1*sol[:, 2] - data).ravel()
    #return (sol[:, 11] - data).ravel()
    
    

In [16]:
initial_conditions = [initS1,initE_0,initE_V, initI_0,initI_V, initR_0,initR_V,initR3,initR4, initN, initD, initI_C]



beta = 0.27940779 
beta1 = 3.2
beta2 = 0.36338425
k1 = 0.25000000
k2 = 0.37000000
deta1 = 0.04859271
deta2 =  1.4886e-04 

gama1 = 0.03534426
gama2 =0.30513419
alpha1=0.01697849
yita1=9.9776e-04
yita2=0.00999327


params['beta'].value = beta
params['beta1'].value = beta1
params['beta2'].value = beta2
params['k1'].value = k1
params['k2'].value = k2
params['deta1'].value = deta1
params['deta2'].value = deta2
params['gama1'].value = gama1
params['gama2'].value = gama2
params['alpha1'].value = alpha1
params['yita1'].value = yita1
params['yita2'].value = yita2
days = 239
tspan = np.arange(0, days, 1)

data = z
#data3=z

In [ ]:
# fit model and find predicted values
result = minimize(error, params, args=(initial_conditions, tspan, data), method='leastsq')
# result = minimize(error, params, args=(initial_conditions, tspan, data), method='leastsq', \
# **{'xtol':1.e-15, 'ftol':1.e-15})

In [ ]:
# display fitted statistics
report_fit(result)